In [1]:
import pandas as pd
movies = pd.read_csv("movies.csv")


In [2]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [3]:
#import python's regular expression library
import re

def clean_title(title):
    #removes characters that are not a space, digit, uppercase or lower case letter
    return re.sub("[^a-zA-Z0-9 ]","", title) 

In [4]:
#create a new column in the data

movies["clean_title"] = movies["title"].apply(clean_title)

In [5]:
movies

#term frequency matrix * #inverse document frequency matrix

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [6]:
#Converts titles to numbers to be used in matrix
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range = (1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [7]:
#write a search function that locates the specified movie using matrices

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np



def search(title):
    
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    #calculates similarity between search term and titles
    similarity = cosine_similarity(query_vec, tfidf).flatten()

    #find the 5 most similar titles to the search term
    indices = np.argpartition(similarity,-5)[-5:]
    results = movies.iloc[indices][::-1]
    return results


In [8]:
search("Toy Story")

,movieId,title,genres,clean_title
3021,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
14813,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
59767,201588,Toy Story 4 (2019),Adventure|Animation|Children|Comedy,Toy Story 4 2019
20497,106022,Toy Story of Terror (2013),Animation|Children|Comedy,Toy Story of Terror 2013


In [9]:
#build an interactive search box
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(value = "Toy Story", description = "Movie Title:", disabled = False)
movie_list = widgets.Output()


def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
    
        if len(title)>5:
            display(search(title))
            
movie_input.observe(on_type, names = 'value')
display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [10]:
#Now build recommendation system based on movie ratings

ratings = pd.read_csv("ratings.csv")

In [11]:
ratings
'''when inspecting the ratings file, we see that each entry has a user id, movie id rating and a time stamp'''

'when inspecting the ratings file, we see that each entry has a user id, movie id rating and a time stamp'

In [12]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [13]:
#Find all the users that liked the same movie as us

In [14]:
#sample movie id for toy story
movie_id = 1

In [15]:
similar_users = ratings[(ratings["movieId"]==movie_id) &(ratings["rating"]>4)]["userId"].unique()

In [16]:
similar_users #an array of all the users who rated toy story 5/5

array([    36,     75,     86, ..., 162527, 162530, 162533], dtype=int64)

In [17]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) &(ratings["rating"]>4)]["movieId"]

In [18]:
similar_user_recs #all the movies that users similar to us also liked

5101            1
5105           34
5111          110
5114          150
5127          260
            ...  
24998854    60069
24998861    67997
24998876    78499
24998884    81591
24998888    88129
Name: movieId, Length: 1358326, dtype: int64

In [19]:
#see how many times each movie appears in similar_user_res as a percentage of total similar_users
similar_user_recs = similar_user_recs.value_counts()/len(similar_users)

#only take the movies with similarity greater than 10%
similar_user_recs = similar_user_recs[similar_user_recs>.1]

In [20]:
similar_user_recs

1        1.000000
318      0.445607
260      0.403770
356      0.370215
296      0.367295
           ...   
953      0.103053
551      0.101195
1222     0.100876
745      0.100345
48780    0.100186
Name: movieId, Length: 113, dtype: float64

In [21]:
#Find how much all users in the dataset like the movies
#Done for the reason that just because a user likes a certain movie doesn't mean it's connected to the movie you searched for

all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"]>4)]

In [22]:
all_users

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
29,1,4973,4.5,1147869080
48,1,7361,5.0,1147880055
72,2,110,5.0,1141416589
76,2,260,5.0,1141417172
...,...,...,...,...
25000062,162541,5618,4.5,1240953299
25000065,162541,5952,5.0,1240952617
25000078,162541,7153,5.0,1240952613
25000081,162541,7361,4.5,1240953484


In [23]:
#find what percentage of all users reccomend each movie
#create a new data frame

all_users_recs = all_users["movieId"].value_counts()/len(all_users["userId"].unique())

In [24]:
all_users_recs

318      0.342220
296      0.284674
2571     0.244033
356      0.235266
593      0.225909
           ...   
551      0.040918
50872    0.039111
745      0.037031
78499    0.035131
2355     0.025091
Name: movieId, Length: 113, dtype: float64

In [25]:
#create a reccomendation score
#join similar_user_recs and all_users_recs by their columns
rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [26]:
rec_percentages

,similar,all
1,1.000000,0.124728
318,0.445607,0.342220
260,0.403770,0.222207
356,0.370215,0.235266
296,0.367295,0.284674
...,...,...
953,0.103053,0.045792
551,0.101195,0.040918
1222,0.100876,0.066877
745,0.100345,0.037031


In [27]:
rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]
rec_percentages = rec_percentages.sort_values("score", ascending= False)

In [28]:
rec_percentages

,similar,all,score
1,1.000000,0.124728,8.017414
3114,0.280648,0.053706,5.225654
2355,0.110539,0.025091,4.405452
78499,0.152960,0.035131,4.354038
4886,0.235147,0.070811,3.320783
...,...,...,...
2858,0.216724,0.167634,1.292845
296,0.367295,0.284674,1.290232
79132,0.166817,0.131384,1.269693
4973,0.142501,0.112405,1.267747


In [29]:
rec_percentages.head(10).merge(movies, left_index = True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.124728,8.017414,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,0.280648,0.053706,5.225654,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
2264,0.110539,0.025091,4.405452,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,Bugs Life A 1998
14813,0.152960,0.035131,4.354038,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
4780,0.235147,0.070811,3.320783,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,Monsters Inc 2001
580,0.216618,0.067513,3.208539,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin 1992
6258,0.228139,0.072268,3.156862,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,Finding Nemo 2003
587,0.179400,0.059977,2.991150,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,Beauty and the Beast 1991
8246,0.203504,0.068453,2.972889,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,Incredibles The 2004
359,0.253411,0.085764,2.954762,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,Lion King The 1994


In [30]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"]==movie_id) &(ratings["rating"]>4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) &(ratings["rating"]>4)]["movieId"]
    #see how many times each movie appears in similar_user_res as a percentage of total similar_users
    similar_user_recs = similar_user_recs.value_counts()/len(similar_users)

    #only take the movies with similarity greater than 10%
    similar_user_recs = similar_user_recs[similar_user_recs>.1]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"]>4)]
    all_users_recs = all_users["movieId"].value_counts()/len(all_users["userId"].unique())
    
    #score them
    rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending= False)
    
    return rec_percentages.head(10).merge(movies, left_index = True, right_on="movieId")[["score", "title", "genres"]]

In [31]:
#create interactive widget

movie_name_input=widgets.Text(value = "Toy Story",
                             description = "Movie Title:",
                             disabled = False)
recommendation_list = widgets.Output()
def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title)>5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))
movie_name_input.observe(on_type, names = "value")  
display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()